In [104]:
# Write a csv file with all the scheduled events
# use spark to read the csv file and create a dataframe
import os
import random
import string
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf, SparkContext
import pandas as pd
import numpy as np

def delete_value_from_list(the_list):

    q1 = np.percentile(the_list, 25)
    q3 = np.percentile(the_list, 75)


    iqr = q3 - q1

    lower_bound = q1 - (iqr * 1)
    upper_bound = q3 + (iqr * 1)

    new_list = [x for x in the_list if x >= lower_bound and x <= upper_bound]

    for _ in range(0, 2):
        var = np.var(new_list)
        best_var = var
        if len(new_list) > 2:
            for i in range(0, len(new_list)):
                test = new_list.copy()
                test.pop(i)
                # get the best variance and keep the index
                if np.var(test) < best_var:
                    best_var = np.var(test)
                    index = i
        if best_var * 2.5 < var:
            new_list.pop(index)

    return np.mean(new_list)



conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


# Get the list of files
files = os.listdir('data')

# create an empty pandas dataframe
df_final = pd.DataFrame(columns=['flow', 'batch_size', 'machine_kafka', 'time', 'nb_messages'])
# Create a new file
for file in files:

    if file.startswith("dat"):
        df = spark.read.csv('data/'+file, header=True)
        print("Load file: "+file)
        df = df.toPandas()
        df['time'] = df['time'].astype(float)
        # if time is > 3 and < 5 replace the value by 4
        df.loc[(df['time'] > 5) & (df['time'] < 100), 'time'] = 5
        # if time is > 100 replace the value by 5
        df.loc[(df['time'] > 100), 'time'] = 7
        # transform the nb_messages column to int
        df['nb_messages'] = df['nb_messages'].astype(int)
        # if nb_messages is < 1000 remove the row
        df = df[df['nb_messages'] > 1000]
        # add the dataframe to the final dataframe if it's empty
        if df_final.empty:
            df_final = df
        else:
            # union the dataframe
            df_final = pd.concat([df_final, df])

# group by flow and batch_size and list each time
df = df_final.groupby(['flow', 'batch_size','machine_kafka'])['time'].apply(list).reset_index()

# for each row, get the average of the time
df['time'] = df['time'].apply(lambda x: delete_value_from_list(x))
# modify the batch_size column values with his value/5632
df['batch_size'] = df['batch_size'].astype(float)
df['batch_size'] = df['batch_size']/5632

# transform the batch_size column and nb_messages to int
df['batch_size'] = df['batch_size'].astype(int)


# order the columns
df = df[['flow', 'batch_size','machine_kafka', 'time']]
df.to_csv('data/'+"merge"+df.machine_kafka[0]+".csv", index=False)
# display the first 5 rows
df.head(10)

Load file: data (1).csv
df_final empty
   flow batch_size machine_kafka      time  nb_messages
0  1600     5632.0       L8as_v3  0.116350        46807
1  1600    16896.0       L8as_v3  0.113554        44101
2  1600    50688.0       L8as_v3  0.137988        39316
4  1600    50688.0       L8as_v3  0.123058        48486
5  1600   152064.0       L8as_v3  0.128201        43776
Load file: data (2).csv
   flow batch_size machine_kafka      time  nb_messages
0  1600     5632.0       L8as_v3  0.108170        46929
1  1600    16896.0       L8as_v3  0.107233        87620
2  1600   152064.0       L8as_v3  0.117545        44131
3  1600   152064.0       L8as_v3  0.115113        43618
4  1600  1368576.0       L8as_v3  0.113108        43787
Load file: data (3).csv
   flow batch_size machine_kafka      time  nb_messages
0  1600     5632.0       L8as_v3  1.069950        46999
1  1600    16896.0       L8as_v3  0.108890        43905
2  1600    50688.0       L8as_v3  0.108004        43402
3  1600   152064.

,flow,batch_size,machine_kafka,time
0,1600,243,L8as_v3,0.129596
1,1600,27,L8as_v3,0.115464
2,1600,3,L8as_v3,0.107958
3,1600,9,L8as_v3,0.105113
4,1600,1,L8as_v3,0.532586
5,2200,243,L8as_v3,0.145543
6,2200,27,L8as_v3,0.135113
7,2200,3,L8as_v3,0.145854
8,2200,81,L8as_v3,0.144859
9,2200,9,L8as_v3,0.140649


In [108]:
# Write a csv file with all the scheduled events
# use spark to read the csv file and create a dataframe
import os
import random
import string
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf, SparkContext
import pandas as pd

conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


# Get the list of files
files = os.listdir('data')

# create an empty pandas dataframe
df_final = pd.DataFrame(columns=['flow', 'batch_size', 'machine_kafka', 'time'])

# Create a new file
for file in files:
    if file.startswith("merge"):
        df = spark.read.csv('data/'+file, header=True)
        print("Load file: "+file)
        df = df.toPandas()
        # if df_final is empty, add the dataframe to it
        # else, union the dataframe
        # add the dataframe to the final dataframe if it's empty
        if df_final.empty:
            df_final = df
        else:
            # union the dataframe
            df_final = pd.concat([df_final, df])

df_final.to_csv('data/'+"super.csv", index=False)

Load file: mergeB2ms.csv
Load file: mergeL8as_v3.csv


In [3]:
# Write a csv file with all the scheduled events
# use spark to read the csv file and create a dataframe
from ast import literal_eval
import os
import random
import string
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf, SparkContext
import pandas as pd
from pyspark.sql.functions import lit

conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


# Get the list of files
files = os.listdir('data')

df_final = 0
# Create a new file
for file in files:

    if file.startswith("super"):
        df = spark.read.csv('data/'+file, header=True)
        print("Load file: "+file)
        # add the dataframe to the final dataframe
        if df_final == 0:
            df_final = df
        else:
            df_final = df_final.union(df)

# remove columns groupID system_prod processor_prod cpu_brand_prod cpu_hz_prod cpu_cores_prod cpu_cores_total_prod ram_total_prod pricePerHour_prod system_cons processor_cons cpu_brand_cons cpu_hz_cons cpu_cores_cons cpu_cores_total_cons ram_total_cons pricePerHour_cons count
df_final = df_final.drop('system_prod', 'processor_prod', 'cpu_brand_prod', 'cpu_hz_prod', 'cpu_cores_prod', 'cpu_cores_total_prod', 'ram_total_prod', 'pricePerHour_prod', 'system_cons', 'processor_cons', 'cpu_brand_cons', 'cpu_hz_cons', 'cpu_cores_cons', 'cpu_cores_total_cons', 'ram_total_cons', 'pricePerHour_cons')

# add column producer and consumer with the values "DS11_V2"

df_final = df_final.withColumn("producer", lit('DS11_V2'))
df_final = df_final.withColumn("consumer", lit('DS11_V2'))

# mixed the dataframe randomly
df_final = df_final.toPandas().sample(frac=1)

# write the dataframe to a csv file
df_final.to_csv('data/'+"merge_all.csv", index=False)



Load file: superds3Opti.csv
